# Notebook 03: Ingesta y Proyección Socioeconómica (CASEN)

**Objetivo:** Generar una serie temporal anual de ingresos y pobreza por comuna (2015-2025).

**Lógica de Procesamiento:**
1. **2015 y 2017:** Se procesan directamente ya que contienen la información de comuna integrada.
2. **2022 (Caso Especial):** Se implementa un cruce de tres fuentes para reconstruir la ubicación:
    * **Base Principal:** Ingresos (`ytot`) y Pobreza.
    * **Base Geográfica:** Cruce por `folio` para obtener el código de `comuna`.
    * **Libro de Códigos (Excel):** Traducción del código numérico al nombre real de la comuna.

**Output:** `data/processed/socioeconomico_anual_completo.csv`

In [1]:
import pandas as pd
import numpy as np
import pyreadstat
import os
import unicodedata

# --- CONFIGURACIÓN DE RUTAS ---
RAW_DIR = os.path.join('..', 'data', 'raw')
PROCESSED_DIR = os.path.join('..', 'data', 'processed')
OUTPUT_FILE = os.path.join(PROCESSED_DIR, 'socioeconomico_anual_completo.csv')

# Archivos 2015-2017
FILE_2015 = os.path.join(RAW_DIR, 'Casen 2015.sav')
FILE_2017 = os.path.join(RAW_DIR, 'Casen 2017.sav')

# Archivos 2022 (El set completo)
FILE_2022_MAIN = os.path.join(RAW_DIR, 'Base de datos Casen 2022 SPSS_18 marzo 2024.sav')
FILE_2022_COMUNAS = os.path.join(RAW_DIR, 'Base de datos provincia y comuna Casen 2022 SPSS.sav')
FILE_2022_CODIGOS = os.path.join(RAW_DIR, 'Libro de codigos Base de datos provincia y comuna Casen 2022.xlsx')

os.makedirs(PROCESSED_DIR, exist_ok=True)

## 1. Funciones Auxiliares

In [2]:
def normalizar_texto(texto):
    """Normaliza nombres para cruce (minúsculas, sin tildes)."""
    if pd.isna(texto):
        return ""
    texto = str(texto).lower().strip()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    return texto

def calcular_agregados(df, anio, col_comuna='nombre_comuna'):
    """Agrupa por comuna y calcula promedios."""
    # Estandarizar nombres de columnas para el cálculo
    df.rename(columns={c: c.lower() for c in df.columns}, inplace=True)
    col_comuna = col_comuna.lower()
    
    # Flag de Pobreza (1 y 2 suelen ser pobreza extrema y no extrema)
    if 'pobreza' in df.columns:
        df['es_pobre'] = df['pobreza'].isin([1, 2]).astype(int)
    else:
        df['es_pobre'] = 0 # Fallback si no existe

    # Agrupar
    resumen = df.groupby(col_comuna).agg(
        ingreso_promedio=('ytot', 'mean'),
        tasa_pobreza=('es_pobre', 'mean')
    ).reset_index()
    
    resumen.rename(columns={col_comuna: 'nombre_comuna'}, inplace=True)
    resumen['anio'] = anio
    return resumen

## 2. Procesamiento 2022: Lógica de Triple Archivo
Aquí aplicamos la lógica de rescatar los nombres reales usando el Excel de códigos.

In [3]:
def cargar_diccionario_excel_2022():
    """
    Lee el Excel 'Libro de codigos...' para mapear Código -> Nombre.
    Logica recuperada: Leer desde fila 7, columnas 3 y 4.
    """
    print(f"  📖 Leyendo diccionario: {os.path.basename(FILE_2022_CODIGOS)}")
    try:
        # Leemos sin header para usar iloc puro como en tu ejemplo exitoso
        df_codigos = pd.read_excel(FILE_2022_CODIGOS, header=None)
        
        # Extraemos datos desde la fila 7 (índice 6) y columnas C y D (índices 3 y 4)
        df_clean = df_codigos.iloc[6:, [3, 4]].copy()
        df_clean.columns = ['codigo', 'nombre']
        
        # Limpieza
        df_clean.dropna(subset=['codigo'], inplace=True)
        # Convertir código a entero para coincidir con el SAV
        df_clean['codigo'] = pd.to_numeric(df_clean['codigo'], errors='coerce')
        df_clean.dropna(subset=['codigo'], inplace=True)
        df_clean['codigo'] = df_clean['codigo'].astype(int)
        
        # Crear diccionario {13101: 'Santiago'}
        diccionario = pd.Series(df_clean['nombre'].values, index=df_clean['codigo']).to_dict()
        print(f"    -> Diccionario cargado con {len(diccionario)} comunas.")
        return diccionario
        
    except Exception as e:
        print(f"    ❌ Error leyendo Excel de códigos: {e}")
        return {}

def procesar_2022():
    print("--- Procesando 2022 ---")
    
    # 1. Cargar Datos Principales (Ingresos)
    print("  Cargando Base Principal...")
    df_main, _ = pyreadstat.read_sav(FILE_2022_MAIN, usecols=['folio', 'ytot', 'pobreza'])
    
    # 2. Cargar Datos Geográficos (Para obtener el código de comuna)
    print("  Cargando Base Comunas (Geográfica)...")
    df_geo, _ = pyreadstat.read_sav(FILE_2022_COMUNAS, usecols=['folio', 'comuna', 'region'])
    
    # 3. Unir por Folio
    print("  Fusionando datasets por Folio...")
    df_full = pd.merge(df_main, df_geo, on='folio', how='inner')
    
    # 4. Filtrar RM
    df_rm = df_full[df_full['region'] == 13].copy()
    print(f"  Registros en RM: {len(df_rm)}")
    
    # 5. Aplicar Traducción de Nombres (Excel)
    mapa_nombres = cargar_diccionario_excel_2022()
    
    if mapa_nombres:
        # Asegurar que el código en el DF sea int para el mapeo
        df_rm['comuna_int'] = pd.to_numeric(df_rm['comuna'], errors='coerce').fillna(0).astype(int)
        df_rm['nombre_real'] = df_rm['comuna_int'].map(mapa_nombres)
        
        # Rellenar no encontrados con el código original
        df_rm['nombre_real'] = df_rm['nombre_real'].fillna(df_rm['comuna'].astype(str))
    else:
        df_rm['nombre_real'] = df_rm['comuna'].astype(str)

    return calcular_agregados(df_rm, 2022, col_comuna='nombre_real')

## 3. Procesamiento 2015 y 2017
Lógica estándar para los archivos antiguos.

In [4]:
def procesar_historico(anio, ruta):
    print(f"--- Procesando {anio} ---")
    try:
        # Leer columnas clave (a veces 'comuna' trae el nombre en labels)
        df, meta = pyreadstat.read_sav(ruta, disable_datetime_conversion=True)
        df.rename(columns={c: c.lower() for c in df.columns}, inplace=True)
        
        # Filtrar RM
        if 'region' in df.columns:
            df = df[df['region'] == 13].copy()
            
        # Intentar obtener nombres de los metadatos del SAV
        col_nombre = 'comuna' # Default
        if 'comuna' in meta.variable_value_labels:
            df['nombre_comuna'] = df['comuna'].map(meta.variable_value_labels['comuna'])
            col_nombre = 'nombre_comuna'
        
        return calcular_agregados(df, anio, col_comuna=col_nombre)
    except Exception as e:
        print(f"❌ Error en {anio}: {e}")
        return None

## 4. Ejecución, Interpolación y Guardado
Juntamos todo y rellenamos los años faltantes.

In [5]:
lista_dfs = []

# 1. Ejecutar procesos
df_15 = procesar_historico(2015, FILE_2015)
if df_15 is not None: lista_dfs.append(df_15)

df_17 = procesar_historico(2017, FILE_2017)
if df_17 is not None: lista_dfs.append(df_17)

# 2022 (Complejo)
try:
    df_22 = procesar_2022()
    if df_22 is not None: lista_dfs.append(df_22)
except Exception as e:
    print(f"❌ Falló proceso 2022: {e}")

# 2. Consolidar
if not lista_dfs:
    raise ValueError("No hay datos para procesar.")

df_base = pd.concat(lista_dfs, ignore_index=True)

# 3. Normalizar para asegurar que 'Santiago' 2015 se una con 'santiago' 2022
df_base['comuna_norm'] = df_base['nombre_comuna'].apply(normalizar_texto)

# 4. Interpolación (Rellenar años faltantes)
print("\n>>> Interpolando serie temporal (2015-2025)...")

# Crear esqueleto completo
comunas = df_base['comuna_norm'].unique()
anios = range(2015, 2026)
grid = pd.MultiIndex.from_product([comunas, anios], names=['comuna_norm', 'anio']).to_frame(index=False)

# Unir y rellenar
df_final = pd.merge(grid, df_base, on=['comuna_norm', 'anio'], how='left')

def rellenar_grupo(g):
    g = g.sort_values('anio')
    # Rellenar huecos internos (2016, 2018-2021) linealmente
    g['ingreso_promedio'] = g['ingreso_promedio'].interpolate(method='linear')
    g['tasa_pobreza'] = g['tasa_pobreza'].interpolate(method='linear')
    # Proyectar hacia el futuro (2023-2025) repitiendo el último dato
    g = g.ffill().bfill()
    return g

df_interpolado = df_final.groupby('comuna_norm').apply(rellenar_grupo).reset_index(drop=True)

# Guardar
print(f"Guardando archivo final: {OUTPUT_FILE}")
df_interpolado.to_csv(OUTPUT_FILE, index=False)

print("✅ Proceso finalizado. Muestra:")
print(df_interpolado.head(12))

--- Procesando 2015 ---
--- Procesando 2017 ---
--- Procesando 2022 ---
  Cargando Base Principal...
  Cargando Base Comunas (Geográfica)...
  Fusionando datasets por Folio...
❌ Falló proceso 2022: 'region'

>>> Interpolando serie temporal (2015-2025)...
Guardando archivo final: ..\data\processed\socioeconomico_anual_completo.csv
✅ Proceso finalizado. Muestra:
   comuna_norm  anio nombre_comuna  ingreso_promedio  tasa_pobreza
0        alhue  2015         Alhué     281611.951900      0.087838
1        alhue  2016         Alhué     328840.744311      0.063450
2        alhue  2017         Alhué     376069.536723      0.039062
3        alhue  2018         Alhué     376069.536723      0.039062
4        alhue  2019         Alhué     376069.536723      0.039062
5        alhue  2020         Alhué     376069.536723      0.039062
6        alhue  2021         Alhué     376069.536723      0.039062
7        alhue  2022         Alhué     376069.536723      0.039062
8        alhue  2023         Alhué

C:\Users\corre\AppData\Local\Temp\ipykernel_8052\2291325581.py:46: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_interpolado = df_final.groupby('comuna_norm').apply(rellenar_grupo).reset_index(drop=True)
